In [27]:
import numpy as np
import pandas as pd
from typing import  List
import datetime

In [28]:
PATH_TO_PICKLE_FILE = '.././Data/stocks.pkl'

In [29]:
import pickle
with open(PATH_TO_PICKLE_FILE, 'rb') as file:
    stocks = pickle.load(file)

In [30]:
data = stocks.copy()

## Compute Scores

For each ticker and date, the procedure computes a full panel of trend, momentum, volatility, oscillator, volume, price–volume, return decomposition, and candlestick features from past OHLCV data, creates a forward return label. After computing these features, the pipeline performs cross-sectional normalization, meaning that for each date, all stocks feature values are first winsorized (extreme values are clipped to the daily 1%/99% quantiles so outliers cannot dominate) and then z-scored across the cross-section (subtract that day’s cross-sectional mean and divide by that day’s cross-sectional standard deviation). This ensures that on each date, all features are on a comparable scale across stocks, are robust to outliers, and can be used reliably in cross-sectional models or trading signals.

We include the Close Price for each observation because we later need it for Triple Barrier Labeling.

In [31]:
def build_ohlc_feature_pipeline(
    df,
    ma_windows=(5, 10, 20, 60),
    ret_windows=(1, 5, 10, 20),
    bb_window=20,
    rsi_window=14,
    stoch_windows=(10, 20),
    aroon_windows=(10, 20),
    adv_window=20,
    atr_window=14,
    pv_window=20,
    winsor_limit=0.01,
    label_horizon=5,
):
    """
    Input df columns (panel, long format):
        ['ticker','PERMNO','Date','DlyVol','DlyClose','DlyLow','DlyHigh','DlyOpen']

    Returns:
        features    : DataFrame indexed by [Date, PERMNO] with cross-sectionally
                      winsorized & z-scored features
        labels      : same index, future C2C return y_{t->t+H*}
        raw_features: same index, un-normalized features (optional for debugging)
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['PERMNO', 'Date'])

    # Remember original cols so we can identify newly created features later
    base_cols = list(df.columns)


    # Step 3–12: per Permno time-series features
    df = (
        df.groupby('PERMNO', group_keys=False)
          .apply(
              _add_per_permno_features,
              ma_windows=ma_windows,
              ret_windows=ret_windows,
              bb_window=bb_window,
              rsi_window=rsi_window,
              stoch_windows=stoch_windows,
              aroon_windows=aroon_windows,
              adv_window=adv_window,
              atr_window=atr_window,
              pv_window=pv_window,
              label_horizon=label_horizon,
          )
    )

    # All newly created columns = feature candidates + label
    new_cols = [c for c in df.columns if c not in base_cols]
    label_col = f'fwd_ret_{label_horizon}'
    feature_cols = [c for c in new_cols if c != label_col]

    ######## Cross-sectional winsorization & z-score by date ###########

    # Put Date, ticker into index for easier groupby
    df = df.set_index(['Date', 'PERMNO']).sort_index()

    features_raw = df[feature_cols].copy()

    # Winsorize per date
    def _winsorize(x):
        lo = x.quantile(winsor_limit)
        hi = x.quantile(1 - winsor_limit)
        return x.clip(lo, hi)

    features_w = features_raw.groupby(level='Date').transform(_winsorize)

    # Z-score per date
    def _zscore(x):
        return (x - x.mean()) / x.std(ddof=0)

    features_z = features_w.groupby(level='Date').transform(_zscore)

    # Drop warm-up rows with any NaN across features or label
    valid = (~features_z.isna().any(axis=1)) & (~df[label_col].isna())
    features_z = features_z[valid]
    labels = df.loc[valid, label_col]

    scores_df = features_z.copy()
    scores_df['label'] = labels
    scores_df["DlyClose"] = df.loc[valid, 'DlyClose']
    
    return scores_df

# Per PERMNO feature builder
def _add_per_permno_features(
    g,
    ma_windows,
    ret_windows,
    bb_window,
    rsi_window,
    stoch_windows,
    aroon_windows,
    adv_window,
    atr_window,
    pv_window,
    label_horizon,
):
    """
    g: DataFrame for a single ticker, sorted by Date.
    Adds OHLC + Volume based features in place and returns g.
    """
    g = g.sort_values('Date').copy()

    O = g['DlyOpen']
    H = g['DlyHigh']
    L = g['DlyLow']
    C = g['DlyClose']
    V = g['DlyVol']

    # ---------- 3. Trend / level: MA, EMA, slopes ----------
    for w in ma_windows:
        ma = C.rolling(w).mean()
        ema = C.ewm(span=w, adjust=False).mean()
        g[f'ma_{w}'] = ma
        g[f'ema_{w}'] = ema
        g[f'slope_ma_{w}'] = ma.diff()  # simple 1-day slope

    # ---------- 4. Momentum: rolling returns & MACD ----------
    # rolling returns over various horizons r_{1,5,10,20}
    for k in ret_windows:
        g[f'ret_{k}'] = C.pct_change(k)

    # MACD (DIF, DEA, MACD, cross flag)
    ema_short = C.ewm(span=12, adjust=False).mean()
    ema_long = C.ewm(span=26, adjust=False).mean()
    dif = ema_short - ema_long
    dea = dif.ewm(span=9, adjust=False).mean()
    macd = 2 * (dif - dea)
    g['macd_dif'] = dif
    g['macd_dea'] = dea
    g['macd'] = macd

    # cross flag: sign change in (DIF - DEA)
    signal = dif - dea
    g['macd_cross'] = (signal * signal.shift(1) < 0).astype(float)

    # ---------- 5. Bands / oscillators ----------
    # Bollinger(20) & %B
    mid = C.rolling(bb_window).mean()
    std = C.rolling(bb_window).std()
    upper = mid + 2 * std
    lower = mid - 2 * std
    g['bb_mid'] = mid
    g['bb_upper'] = upper
    g['bb_lower'] = lower
    g['bb_pctB'] = (C - lower) / (upper - lower)

    # RSI
    delta = C.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.rolling(rsi_window).mean()
    roll_down = down.rolling(rsi_window).mean()
    rs = roll_up / roll_down
    g['rsi'] = 100 - 100 / (1 + rs)

    # Stochastic RSV_w
    for w in stoch_windows:
        low_min = L.rolling(w).min()
        high_max = H.rolling(w).max()
        g[f'rsv_{w}'] = (C - low_min) / (high_max - low_min)

    # Aroon Up / Down (simple rolling implementation; can be optimized)
    def _aroon_up(x):
        # position of last max in window, counted from 0
        return (len(x) - 1 - np.argmax(x.values)) / (len(x) - 1) * 100

    def _aroon_down(x):
        return (len(x) - 1 - np.argmin(x.values)) / (len(x) - 1) * 100

    for w in aroon_windows:
        g[f'aroon_up_{w}'] = H.rolling(w).apply(_aroon_up, raw=False)
        g[f'aroon_down_{w}'] = L.rolling(w).apply(_aroon_down, raw=False)

    # ---------- 6. Ranges / gaps / candlestick ratios ----------
    # (H − L)/C
    g['range_hl_over_c'] = (H - L) / C

    # ATR (using classic True Range)
    prev_close = C.shift(1)
    tr = pd.concat(
        [
            H - L,
            (H - prev_close).abs(),
            (L - prev_close).abs(),
        ],
        axis=1,
    ).max(axis=1)
    g['atr'] = tr.rolling(atr_window).mean()

    # gaps: O − C_{t-1}, C − C_{t-1}
    g['gap_oc_prev'] = O - prev_close
    g['gap_cc_prev'] = C - prev_close

    # candlestick shadow ratios
    upper_shadow = H - np.maximum(O, C)
    lower_shadow = np.minimum(O, C) - L
    body = (C - O).abs()
    g['shadow_upper_ratio'] = upper_shadow / (body + 1e-12)
    g['shadow_lower_ratio'] = lower_shadow / (body + 1e-12)

    # ---------- 7. Point-in-day returns ----------
    # C2C r_t
    g['ret_c2c'] = C.pct_change()
    # CO: close vs open in same day
    g['ret_co'] = C / O - 1
    # OC: open vs previous close (overnight gap)
    g['ret_oc'] = O / prev_close - 1

    # ---------- 8. Volume set ----------
    g['vol'] = V
    g['adv'] = V.rolling(adv_window).mean()
    g['dvol'] = V.pct_change()
    # volume z-score within each ticker (time-series)
    g['vol_z'] = (V - g['adv']) / V.rolling(adv_window).std(ddof=0)

    # ---------- 9. Turnover (proxy using relative volume) ----------
    g['turnover_proxy'] = V / g['adv']

    # ---------- 10. VWAP (proxy) ----------
    vwap = (O + H + L + C) / 4.0
    g['vwap_proxy'] = vwap

    # ---------- 11. Price–Volume Divergence ----------
    # correlation between VWAP and Volume over a lookback window
    g['pv_corr'] = vwap.rolling(pv_window).corr(V)
    g['pv_divergence'] = -g['pv_corr']   # feature is −ρ

    # ---------- 12. Label: forward C2C return over horizon H* ----------
    g[f'fwd_ret_{label_horizon}'] = C.shift(-label_horizon) / C - 1

    return g


In [32]:
df_scores = build_ohlc_feature_pipeline(data)

/var/folders/zb/_v_3sbq938jftd_b07f5kpfm0000gn/T/ipykernel_90952/3306131521.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [33]:
df_scores

ma_5     ema_5  slope_ma_5     ma_10    ema_10  \
Date       PERMNO                                                       
2013-04-01 10026   1.171313  1.161774    0.541080  1.141680  1.141524   
           10145   1.149054  1.142536   -0.859211  1.146074  1.138618   
           10158  -0.801484 -0.802927    0.206227 -0.806610 -0.801662   
           10207  -0.814302 -0.813424   -0.077890 -0.813643 -0.812927   
           10252  -0.077493 -0.081945   -0.362007 -0.078820 -0.081338   
...                     ...       ...         ...       ...       ...   
2023-12-21 93380  -0.368517 -0.368925    0.673617 -0.373438 -0.370163   
           93415  -0.333871 -0.332995   -0.029628 -0.332091 -0.330406   
           93419  -0.603996 -0.605305   -0.068915 -0.609252 -0.609151   
           93427   1.105182  1.094618    1.702947  1.054806  1.067444   
           93429   0.964248  0.971734    1.435792  0.996701  0.996126   

                   slope_ma_10     ma_20    ema_20  slope_ma_20     ma_60  \
Date       PERMNO                                                           
2013-04-01 10026      1.749915  1.092093  1.109808     1.983100  1.020783   
           10145      0.566247  1.117035  1.122651     1.594787  1.055711   
           10158     -0.207468 -0.795113 -0.794578    -0.748634 -0.753679   
           10207     -0.040022 -0.811231 -0.811746    -0.369352 -0.801228   
           10252     -0.028474 -0.085185 -0.082850    -0.139074 -0.088104   
...                        ...       ...       ...          ...       ...   
2023-12-21 93380     -0.090457 -0.369716 -0.366960    -0.504708 -0.364130   
           93415     -0.549918 -0.321932 -0.325615    -0.391923 -0.318712   
           93419     -0.320187 -0.618376 -0.615913    -0.152698 -0.632162   
           93427      3.346722  1.010483  1.043644     2.085799  1.065224   
           93429     -0.671368  1.059082  1.033782    -0.922466  1.077275   

                   ...       vol       adv      dvol     vol_z  \
Date       PERMNO  ...                                           
2013-04-01 10026   ... -0.449389 -0.453514  0.669874  0.502964   
           10145   ...  1.285469  0.981474  0.136404  0.078073   
           10158   ... -0.411934 -0.404023 -0.343869 -0.318661   
           10207   ... -0.457803 -0.453907  1.305792 -0.616476   
           10252   ... -0.437524 -0.445881  0.222168  0.348070   
...                ...       ...       ...       ...       ...   
2023-12-21 93380   ... -0.116259 -0.236797  1.154491  0.825230   
           93415   ... -0.485686 -0.518971 -1.034074 -0.880380   
           93419   ...  0.526555  0.665130 -0.541223 -0.493217   
           93427   ... -0.403387 -0.430691 -0.773993 -0.262785   
           93429   ... -0.254847 -0.229511 -0.340201 -0.755325   

                   turnover_proxy  vwap_proxy   pv_corr  pv_divergence  \
Date       PERMNO                                                        
2013-04-01 10026         0.290945    1.171328 -0.817272       0.817272   
           10145        -0.002715    1.134350 -0.045139       0.045139   
           10158        -0.525320   -0.802024 -2.006186       2.006186   
           10207        -0.580649   -0.813009 -1.320348       1.320348   
           10252         0.388319   -0.088090  0.144052      -0.144052   
...                           ...         ...       ...            ...   
2023-12-21 93380         0.758403   -0.366640 -0.337685       0.337685   
           93415        -0.998471   -0.333002  1.030022      -1.030022   
           93419        -0.274511   -0.603613  0.499878      -0.499878   
           93427        -0.202414    1.110732  1.394271      -1.394271   
           93429        -0.433524    0.962727 -0.779382       0.779382   

                      label  DlyClose  
Date       PERMNO                      
2013-04-01 10026  -0.022231     75.12  
           10145  -0.010494     74.33  
           10158  -0.019499      7.18  
           10207  -0.020349      6.88  
         

In [34]:
df_scores_flat = df_scores.reset_index()

In [35]:
df_scores_flat

,Date,PERMNO,ma_5,ema_5,slope_ma_5,ma_10,ema_10,slope_ma_10,ma_20,ema_20,...,vol,adv,dvol,vol_z,turnover_proxy,vwap_proxy,pv_corr,pv_divergence,label,DlyClose
0,2013-04-01,10026,1.171313,1.161774,0.541080,1.141680,1.141524,1.749915,1.092093,1.109808,...,-0.449389,-0.453514,0.669874,0.502964,0.290945,1.171328,-0.817272,0.817272,-0.022231,75.12
1,2013-04-01,10145,1.149054,1.142536,-0.859211,1.146074,1.138618,0.566247,1.117035,1.122651,...,1.285469,0.981474,0.136404,0.078073,-0.002715,1.134350,-0.045139,0.045139,-0.010494,74.33
2,2013-04-01,10158,-0.801484,-0.802927,0.206227,-0.806610,-0.801662,-0.207468,-0.795113,-0.794578,...,-0.411934,-0.404023,-0.343869,-0.318661,-0.525320,-0.802024,-2.006186,2.006186,-0.019499,7.18
3,2013-04-01,10207,-0.814302,-0.813424,-0.077890,-0.813643,-0.812927,-0.040022,-0.811231,-0.811746,...,-0.457803,-0.453907,1.305792,-0.616476,-0.580649,-0.813009,-1.320348,1.320348,-0.020349,6.88
4,2013-04-01,10252,-0.077493,-0.081945,-0.362007,-0.078820,-0.081338,-0.028474,-0.085185,-0.082850,...,-0.437524,-0.445881,0.222168,0.348070,0.388319,-0.088090,0.144052,-0.144052,-0.020997,31.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655554,2023-12-21,93380,-0.368517,-0.368925,0.673617,-0.373438,-0.370163,-0.090457,-0.369716,-0.366960,...,-0.116259,-0.236797,1.154491,0.825230,0.758403,-0.366640,-0.337685,0.337685,0.049730,35.19
2655555,2023-12-21,93415,-0.333871,-0.332995,-0.029628,-0.332091,-0.330406,-0.549918,-0.321932,-0.325615,...,-0.485686,-0.518971,-1.034074,-0.880380,-0.998471,-0.333002,1.030022,-1.030022,0.064242,37.67
2655556,2023-12-21,93419,-0.603996,-0.605305,-0.068915,-0.609252,-0.609151,-0.320187,-0.618376,-0.615913,...,0.526555,0.665130,-0.541223,-0.493217,-0.274511,-0.603613,0.499878,-0.499878,0.041387,8.94
2655557,2023-12-21,93427,1.105182,1.094618,1.702947,1.054806,1.067444,3.346722,1.010483,1.043644,...,-0.403387,-0.430691,-0.773993,-0.262785,-0.202414,1.110732,1.394271,-1.394271,-0.005694,191.42


In [36]:
with open('.././Data/feature_scores.pkl', 'wb') as file:
    pickle.dump(df_scores, file)

In [37]:
with open(".././Data/feature_scores_flat.pkl", "wb") as file:
    pickle.dump(df_scores_flat, file)